# ChatBot

In [9]:
#Importing libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD
import json
import random

Tokenization

In [10]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

Lowering & Stemming

In [11]:
stemmer = PorterStemmer()
def stem(word):
    return stemmer.stem(word.lower())

Bag of Word

In [12]:
def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence ]
    bag = np.zeros(len(all_words), dtype= np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

Reading Tags, patterns from json file & applying tokenization ans stemming

In [13]:
with open('intents.json','r') as f:
    intents = json.load(f)
    
all_words = []
tags =[]
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w,tag))
ignore_words = ['!','?','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))    # Set removes duplicate
tags = sorted(tags)

Creating training data using Bag of Words

In [14]:
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence,all_words)
    X_train.append(bag)
    output_words = [0]*len(tags)
    output_words = list(output_words)
    output_words[tags.index(tag)] = 1
    y_train.append(output_words)
    
X_train = np.array(X_train)
y_train = np.array(y_train)

## Modeling 

In [19]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

2022-05-05 18:11:11.368671: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-05 18:11:11.371337: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [20]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [21]:
#fitting 
chatbot_model = model.fit(X_train, y_train, epochs=200, batch_size=5, verbose=1)

Train on 35 samples
Epoch 1/200
35/35 [==============================] - 1s 21ms/sample - loss: 2.2469 - accuracy: 0.0571
Epoch 2/200
35/35 [==============================] - 0s 424us/sample - loss: 2.1409 - accuracy: 0.1429
Epoch 3/200
35/35 [==============================] - 0s 561us/sample - loss: 2.0697 - accuracy: 0.2286
Epoch 4/200
35/35 [==============================] - 0s 525us/sample - loss: 1.9411 - accuracy: 0.2571
Epoch 5/200
35/35 [==============================] - 0s 342us/sample - loss: 1.9463 - accuracy: 0.3429
Epoch 6/200
35/35 [==============================] - 0s 268us/sample - loss: 1.8553 - accuracy: 0.4000
Epoch 7/200
35/35 [==============================] - 0s 347us/sample - loss: 1.8039 - accuracy: 0.4000
Epoch 8/200
35/35 [==============================] - 0s 452us/sample - loss: 1.7022 - accuracy: 0.4000
Epoch 9/200
35/35 [==============================] - 0s 382us/sample - loss: 1.6105 - accuracy: 0.4857
Epoch 10/200
35/35 [==============================] - 

35/35 [==============================] - 0s 371us/sample - loss: 0.1486 - accuracy: 0.9714
Epoch 80/200
35/35 [==============================] - 0s 391us/sample - loss: 0.0164 - accuracy: 1.0000
Epoch 81/200
35/35 [==============================] - 0s 427us/sample - loss: 0.0331 - accuracy: 1.0000
Epoch 82/200
35/35 [==============================] - 0s 340us/sample - loss: 0.0626 - accuracy: 1.0000
Epoch 83/200
35/35 [==============================] - 0s 378us/sample - loss: 0.1395 - accuracy: 0.9714
Epoch 84/200
35/35 [==============================] - 0s 396us/sample - loss: 0.1153 - accuracy: 1.0000
Epoch 85/200
35/35 [==============================] - 0s 333us/sample - loss: 0.0294 - accuracy: 1.0000
Epoch 86/200
35/35 [==============================] - 0s 364us/sample - loss: 0.0250 - accuracy: 1.0000
Epoch 87/200
35/35 [==============================] - 0s 348us/sample - loss: 0.0787 - accuracy: 1.0000
Epoch 88/200
35/35 [==============================] - 0s 344us/sample - loss:

35/35 [==============================] - 0s 329us/sample - loss: 0.0043 - accuracy: 1.0000
Epoch 158/200
35/35 [==============================] - 0s 309us/sample - loss: 0.0629 - accuracy: 0.9714
Epoch 159/200
35/35 [==============================] - 0s 325us/sample - loss: 0.0324 - accuracy: 0.9714
Epoch 160/200
35/35 [==============================] - 0s 324us/sample - loss: 0.0192 - accuracy: 1.0000
Epoch 161/200
35/35 [==============================] - 0s 312us/sample - loss: 0.0560 - accuracy: 0.9714
Epoch 162/200
35/35 [==============================] - 0s 293us/sample - loss: 0.0072 - accuracy: 1.0000
Epoch 163/200
35/35 [==============================] - 0s 295us/sample - loss: 0.0124 - accuracy: 1.0000
Epoch 164/200
35/35 [==============================] - 0s 307us/sample - loss: 0.0376 - accuracy: 1.0000
Epoch 165/200
35/35 [==============================] - 0s 306us/sample - loss: 0.0063 - accuracy: 1.0000
Epoch 166/200
35/35 [==============================] - 0s 352us/sampl

Saving Files

In [23]:
import pickle
model.save('model.h5', chatbot_model)
pickle.dump(all_words, open('all_words.pkl','wb'))
pickle.dump(tags, open('tags.pkl','wb'))

Creating BOW for User Inputs

In [24]:
def user_input(sentence):    
    # Tokenization
    sentence = tokenize(sentence)
    # lowering and stemming
    # Generating Bag of Words
    bow = bag_of_words(sentence, all_words)
    return np.array(bow)

Response Model

In [56]:
THRESHOLD=0.25
def response(sentence):
    p = user_input(sentence)
    results = model.predict(np.array([p]))[0]
  # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>THRESHOLD]
  # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((tags[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

In [63]:
chatbot("do you work weekends")

We're open every day 9am-9pm


In [58]:
def chatbot(sentence):
    result = response(sentence)
    if result:
        while result:

            for i in intents['intents']:
      # find a tag matching the first result
                if i['tag'] == result[0][0]:
        # a random response from the intent
                    return print(random.choice(i['responses']))
    else:
        print('Please enter a Valid Response')